###  Lab Exercise #5 Code

In [ ]:
# Importing necessary library
import pandas as pd
import numpy as np
from dateutil.parser import parse
from datetime import datetime

In [ ]:
# Loading and Reading CSV file 
transactions = pd.read_csv('inconsistent_transactions.csv') 
#transactions


### Creating a copy of the original transactions data

In [ ]:

modified_data = transactions.copy()
modified_data

### Extending the range of transaction IDS

In [ ]:
modified_data['transaction_id'] = modified_data['transaction_id'].str.replace(r'^T','T0', regex=True)
modified_data

### Enforcing a single Product ID format

In [ ]:
pd.reset_option('display.max_rows', None)
modified_data['product_id'] = modified_data['product_id'].str.replace(' ','').str.replace('-','').str.replace('p','P')
modified_data

### Changing the price format


In [ ]:
modified_data['price'] = modified_data['price'].str.replace('$','')
modified_data['price'] = modified_data['price'].astype(float).round(2)
modified_data

### Formatting the timestamp


In [ ]:
def new_timestamp(row):
    try:
        if '-' in row and row.count('-') == 2:
            try:
                ts = datetime.strptime(row, '%d-%m-%Y')
                return ts.strftime('%Y-%m-%d') 
            except ValueError:
                pass  
        
        ts = parse(row)
        return ts.strftime('%Y-%m-%d')
    
    except ValueError:
        return row  

modified_data['timestamp'] = modified_data['timestamp'].apply(new_timestamp)
modified_data = modified_data.sort_values(by='timestamp')


### Reformatting the TransactionIDs in chronological order

In [ ]:
modified_data['transaction_id'] = ['T{:05d}'.format(i) for i in range(len(modified_data))]
modified_data

### Computing the unit prices

In [ ]:
## Create a dataframe for rows with complete column values
complete_transactions = modified_data.dropna()
unique_products = complete_transactions.copy()


In [ ]:
## Computation for unit price
unique_products['unit_price'] = (unique_products['price'] / unique_products['quantity'])


In [ ]:
unique_products['max_unit_price'] = unique_products.groupby('product_id')['unit_price'].transform('max')
unique_products = unique_products.drop_duplicates(subset='product_id')
unique_products = unique_products.drop(columns=['transaction_id','quantity','price','timestamp','unit_price'])
unique_products

### Removing rows with null values for both quantity and price

In [ ]:
modified_data = modified_data.dropna(subset=['quantity','price'], how='all')

### Computing for the missing unit prices and quantities

In [ ]:
pd.set_option('display.max_rows',None)
modified_data = modified_data.merge(unique_products, on='product_id', how='left')


In [ ]:
modified_data['price'] = modified_data['price'].fillna((modified_data['max_unit_price'] * modified_data['quantity']).round(2))
modified_data['quantity'] = modified_data['quantity'].fillna((modified_data['price'] / modified_data['max_unit_price']).apply(np.floor))
modified_data['quantity'] = modified_data['quantity'].replace(0, 1)



In [ ]:
# Drop max_unit_price to maintain original data format
modified_data = modified_data.drop(columns=['max_unit_price'])


In [ ]:
# Change quantity to int
modified_data['quantity'] = modified_data['quantity'].astype(int)
modified_data